## TRABAJO DE TITULACIÓN   - Grupo 8

 * Carlos Guzmán
 * Diego Salazar
 * Jonathan Barrionuevo
 * Nicole Molina

### Inicialización del Proyecto


## Configuración del Entorno de Trabajo con Docker

En esta primera parte, vamos a crear y configurar un entorno de trabajo para ejecutar modelos de Machine Learning y procesamiento de datos en un contenedor Docker. El proceso está dividido en dos pasos clave:

### 1. Construcción de la Imagen Docker

El primer paso es crear una **imagen Docker** usando un archivo `Dockerfile`. La imagen actúa como una "plantilla" o "receta" que define todas las dependencias, configuraciones y bibliotecas necesarias para nuestro proyecto. En este caso, estamos usando CUDA, PyTorch, Jupyter Notebook, y otras bibliotecas de procesamiento de datos.

- **Dockerfile**: El archivo `Dockerfile` incluye instrucciones para:
  - Establecer una imagen base compatible con CUDA.
  - Instalar Python y otras dependencias necesarias.
  - Crear un entorno virtual dentro del contenedor.
  - Instalar bibliotecas específicas como PyTorch, NLTK, spaCy, Transformers, y Jupyter Notebook.
  - Configurar el entorno para que esté listo para ejecutar scripts de Machine Learning y Deep Learning.

- **Construcción de la Imagen**: Para crear la imagen, ejecutamos el siguiente comando en la terminal:
  ```bash
  docker build -t cudarnn:latest .

  Esto genera una imagen llamada cudarnn:latest que contiene todo el entorno y las configuraciones definidas en el Dockerfile.

### 2. Ejecución del Contenedor desde el Notebook

Una vez que hemos creado la imagen `cudarnn:latest`, el siguiente paso es **lanzar un contenedor** basado en esta imagen. Este paso lo hacemos directamente desde el notebook (`ipynb`) utilizando un script en Python con `subprocess`.

Este script ejecuta un comando `docker run` que:

*   **Lanza un contenedor** desde la imagen `cudarnn:latest`.
*   **Configura volúmenes** para montar carpetas locales (como `Script` y `Dataset`), permitiendo que el contenedor acceda a archivos en nuestro sistema.
*   **Mapea puertos** para acceder a Jupyter Notebook desde el navegador (usando el puerto 8888).
*   **Asigna GPU** (si está disponible) para aprovechar el soporte de CUDA.
*   **Inicia Jupyter Notebook** dentro del contenedor, permitiendo trabajar en un entorno interactivo a través del navegador.

De esta manera, el notebook actúa como una **interfaz de usuario** para lanzar y trabajar dentro del contenedor Docker, aprovechando el entorno consistente y controlado que hemos creado con la imagen.

> **Nota importante**: Antes de ejecutar el script, asegúrate de **cambiar las rutas absolutas** en el código del notebook (`script_path` y `dataset_path`) para que apunten a las ubicaciones correctas en tu sistema de archivos local. Esto permitirá que el contenedor monte correctamente las carpetas `Script` y `Dataset` dentro del entorno de Docker. Aquí tienes un ejemplo de cómo deberían verse las rutas:
> 
> ```python
> script_path = "C:/Users/DELL/Desktop/M_ML_DL_CUDA_RNN_/Script"  # Ruta completa a Script
> dataset_path = "C:/Users/DELL/Desktop/M_ML_DL_CUDA_RNN_/Dataset"  # Ruta completa a Dataset
> ```
> 
> Asegúrate de ajustar estas rutas de acuerdo a la ubicación de las carpetas en tu máquina. Estas carpetas montadas te permitirán acceder a tus archivos de scripts y datos desde Jupyter Notebook dentro del contenedor.

Una vez que el contenedor esté en ejecución, el script proporcionará una URL con un token de acceso para abrir Jupyter Notebook en tu navegador. Copia y pega esta URL en el navegador para acceder a tu entorno de trabajo dentro del contenedor Docker.


### Acceso a Jupyter Notebook en Docker

Una vez que el contenedor se ha iniciado exitosamente, sigue estos pasos para acceder a Jupyter Notebook:

1. **Obtener el Token de Autenticación**:
   - Abre una terminal o línea de comandos separada.
   - Ejecuta el siguiente comando para ver los logs del contenedor y obtener el token de Jupyter Notebook:
     ```bash
     docker logs nombre_del_contenedor
     ```
   - Reemplaza `nombre_del_contenedor` con el nombre que hayas asignado al contenedor (por ejemplo, `tourism-classificator`).
   - Busca en los logs una línea que contenga una URL similar a:
     ```
     http://127.0.0.1:8888/?token=tu_token_aqui
     ```
   - Copia el valor del token (lo que está después de `?token=`).

2. **Ingresar el Token en el Script**:
   - Vuelve al notebook de configuración (`initial_conf.ipynb`) donde se está ejecutando el script de configuración.
   - Cuando el script te pida el token, pégalo y presiona Enter.

3. **Abrir Jupyter Notebook en el Navegador**:
   - Después de ingresar el token, el script te mostrará una URL completa para acceder a Jupyter Notebook.
   - Copia y pega esta URL en tu navegador para abrir el entorno de Jupyter Notebook.

4. **Comienza a Trabajar en el Entorno de Jupyter Notebook**:
   - Dentro de Jupyter Notebook, podrás acceder a las carpetas `Script` y `Dataset` que configuraste en el contenedor.
   - Puedes abrir y trabajar con notebooks directamente en el navegador en un entorno controlado de Docker.

> **Nota**: Si tienes algún problema con el token, puedes volver a ejecutar el comando `docker logs nombre_del_contenedor` para obtenerlo nuevamente.


In [ ]:
import subprocess
import time
import os

# Pedir al usuario el nombre de la imagen y el nombre del contenedor
image_name = input("Ingresa el nombre de la imagen Docker: ")
container_name = input("Dale un nombre al contenedor: ")

# Rutas absolutas en tu sistema para las carpetas Script y Dataset
script_path = "C:/Users/DELL/Desktop/UIDE/Tesis/Repositorio/M_ML_DL_CUDA_RNN_\Script"  # Ajusta la ruta
dataset_path = "C:/Users/DELL/Desktop/UIDE/Tesis/Repositorio/M_ML_DL_CUDA_RNN_\Dataset"  # Ajusta la ruta

# Construir el comando Docker en modo detached (`-d`)
command = [
    "docker", "run", "--gpus", "all", "-d", "-p", "8888:8888",
    "-v", f"{script_path}:/workspace/Script",
    "-v", f"{dataset_path}:/workspace/Dataset",
    "--name", container_name,
    image_name,
    "/opt/venv/bin/jupyter", "notebook", "--ip=0.0.0.0", "--port=8888", "--no-browser", "--allow-root"
]

# Ejecutar el comando Docker
try:
    # Iniciar el contenedor
    result = subprocess.run(command, check=True, text=True, capture_output=True)
    print("Contenedor iniciado con éxito.")
       
    # Solicitar el token al usuario
    user_token = input("Por favor, ingresa el token de Jupyter Notebook: ")
    jupyter_url = f"http://127.0.0.1:8888/?token={user_token}"
    
    # Mostrar la URL final con el token incluido
    print(f"\nAccede a Jupyter Notebook en tu navegador usando esta URL:\n{jupyter_url}")
except subprocess.CalledProcessError as e:
    print("Error al iniciar el contenedor:")
    print("STDOUT:", e.stdout)
    print("STDERR:", e.stderr)

Contenedor iniciado con éxito.
